In [1]:
import requests, pandas as pd
import time
from pathlib import Path
from typing import Optional


# Crawl dữ liệu và lấy dữ liệu về máy

In [2]:
# tạo thư mục lưu trữ dữ liệu thô
OUT = Path(r"C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw")
OUT.mkdir(parents=True, exist_ok=True)

# Tải dữ liệu thành công và lưu thành file csv

In [3]:
# Địa chỉ API của CafeF để lấy dữ liệu lịch sử giá chứng khoán
BASE = "https://cafef.vn/du-lieu/Ajax/PageNew/DataHistory/PriceHistory.ashx"


def fetch_vnindex_cafef_raw(
    symbol: str = "VNINDEX",   # Mã chứng khoán cần tải (mặc định là VNINDEX)
    pagesize: int = 5000,      # Số lượng bản ghi trên mỗi trang (CafeF giới hạn, thường 5000)
    pause: float = 0.2,        # Thời gian tạm dừng giữa 2 lần gọi API để tránh bị chặn
    out_dir: Path = OUT        # Thư mục lưu kết quả CSV (được định nghĩa ở nơi khác)
) -> pd.DataFrame:
    """
    Hàm này tải dữ liệu lịch sử giá của một mã chứng khoán từ CafeF.vn.
    Dữ liệu được tải từng trang, ghép lại thành DataFrame và lưu ra CSV.
    Trả về: pandas.DataFrame chứa toàn bộ dữ liệu thô.
    """

    # Header giả lập trình duyệt thật, giúp tránh bị server từ chối (403 Forbidden)
    headers = {"User-Agent": "Mozilla/5.0"}

    # Khởi tạo danh sách chứa các dòng dữ liệu và biến đếm trang
    rows, page = [], 1

    # ========== VÒNG LẶP TẢI DỮ LIỆU ========== #
    while True:
        # Tham số truy vấn gửi lên API CafeF
        params = {
            "Symbol": symbol,       # Mã chứng khoán (VD: VNINDEX, VNM, HPG...)
            "StartDate": "",        # Không giới hạn ngày bắt đầu
            "EndDate": "",          # Không giới hạn ngày kết thúc
            "PageIndex": page,      # Trang hiện tại (bắt đầu từ 1)
            "PageSize": pagesize    # Số bản ghi mỗi trang
        }

        # Gửi yêu cầu HTTP GET đến API
        r = requests.get(BASE, params=params, headers=headers, timeout=30)

        # Nếu request thất bại (ví dụ lỗi 404, 500...), lệnh này sẽ dừng và báo lỗi ngay
        r.raise_for_status()

        # Chuyển phản hồi JSON thành dict Python
        payload = r.json()

        # Lấy phần dữ liệu chính từ payload (nằm trong payload["Data"]["Data"])
        data = (payload.get("Data") or {}).get("Data") or []

        # Nếu không còn dữ liệu (trang trống) → thoát vòng lặp
        if not data:
            break

        # Gộp dữ liệu của trang hiện tại vào danh sách tổng
        rows.extend(data)

        # Nếu số dòng nhỏ hơn pagesize, nghĩa là đã đến trang cuối cùng → dừng lại
        if len(data) < pagesize:
            break

        # Tăng số trang lên 1 và nghỉ 0.2 giây để tránh bị server chặn
        page += 1
        time.sleep(pause)

    # ========== TẠO DATAFRAME VÀ LƯU FILE ========== #

    # Chuyển toàn bộ danh sách dữ liệu sang DataFrame pandas
    df = pd.DataFrame(rows)

    # Xác định đường dẫn xuất file CSV (VD: data_raw/VNINDEX_CafeF_RAW.csv)
    out_path = out_dir / f"{symbol}_CafeF_RAW.csv"

    # Lưu DataFrame thành file CSV với encoding hỗ trợ tiếng Việt
    df.to_csv(out_path, index=False, encoding="utf-8-sig")

    # In thông báo ra màn hình: tên mã, số dòng, và đường dẫn file lưu
    print(f"✅ {symbol}: {len(df)} dòng | Đã lưu → {out_path}")

    # Trả về DataFrame để có thể dùng tiếp trong chương trình
    return df

In [4]:
df_raw = fetch_vnindex_cafef_raw()

✅ VNINDEX: 6144 dòng | Đã lưu → C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\data_raw\VNINDEX_CafeF_RAW.csv
